# Import Boundary Conditions from PINACLES and write in AMR-Wind Format

In [1]:
import numpy as np
from pathlib import Path
import h5py
import matplotlib.pyplot as plt
import os
import re
from datetime import timedelta
from scipy import interpolate
from netCDF4 import Dataset

## Extract the PINACLES file names and times from the files

In [2]:
loc = Path('/scratch/ckaul/dycoms_planes_dry_10m/fields2d/')

def get_seconds(filename):
    match = re.match(r"(\d{2})d-(\d{2})h-(\d{2})m-(\d{2})s-(\d{3})ms\.h5", filename)
    if match:
        d, h, m, s, ms = map(int, match.groups())
        return timedelta(days=d, hours=h, minutes=m, seconds=s, milliseconds=ms).total_seconds()

# The names of the files sorted
time_files = sorted(
        [f for f in os.listdir(loc) if f.endswith('.h5') and get_seconds(f) is not None]
    )

# time_data = {
#     f: get_seconds(f) for f in os.listdir(loc) if f.endswith('.h5') and get_seconds(f) is not None
# }

# Print results
# for fname, sec in time_data.items():
#     print(f"{fname}: {sec} seconds")

times_in_seconds = np.array([
    get_seconds(f) for f in time_files
])

#print(time_files)
print(times_in_seconds)

[0.000e+00 1.000e+01 2.000e+01 ... 2.158e+04 2.159e+04 2.160e+04]


## Extract the arrays of velocity data from PINACLES in numpy format

In [3]:
# Open the first file to get the shape of the data arrays (assuming all files have the same shape)
with h5py.File(os.path.join(loc, time_files[0]), "r") as f:
     print(list(f.keys())) 


# Determine the number of time steps
num_files = len(time_files)
print(num_files)

['IWP', 'LWP', 'RAINNC', 'RAINNCV', 'T', 'T_105.0', 'T_205.0', 'T_305.0', 'T_5.0', 'T_x_1800', 'T_x_4800', 'T_x_5800', 'T_x_800', 'T_y_1800', 'T_y_4800', 'T_y_5800', 'T_y_800', 'X', 'Y', 'Z', 'cloud_base', 'cloud_top', 'pseudo-albedo', 'qc_105.0', 'qc_205.0', 'qc_305.0', 'qc_5.0', 'qc_x_1800', 'qc_x_4800', 'qc_x_5800', 'qc_x_800', 'qc_y_1800', 'qc_y_4800', 'qc_y_5800', 'qc_y_800', 'qi1_105.0', 'qi1_205.0', 'qi1_305.0', 'qi1_5.0', 'qi1_x_1800', 'qi1_x_4800', 'qi1_x_5800', 'qi1_x_800', 'qi1_y_1800', 'qi1_y_4800', 'qi1_y_5800', 'qi1_y_800', 'qib1_105.0', 'qib1_205.0', 'qib1_305.0', 'qib1_5.0', 'qib1_x_1800', 'qib1_x_4800', 'qib1_x_5800', 'qib1_x_800', 'qib1_y_1800', 'qib1_y_4800', 'qib1_y_5800', 'qib1_y_800', 'qir1_105.0', 'qir1_205.0', 'qir1_305.0', 'qir1_5.0', 'qir1_x_1800', 'qir1_x_4800', 'qir1_x_5800', 'qir1_x_800', 'qir1_y_1800', 'qir1_y_4800', 'qir1_y_5800', 'qir1_y_800', 'qnc_105.0', 'qnc_205.0', 'qnc_305.0', 'qnc_5.0', 'qnc_x_1800', 'qnc_x_4800', 'qnc_x_5800', 'qnc_x_800', 'qnc_y_

In [4]:
#### Collecting xlo data: For yz plane

datafile = os.path.join(loc, time_files[0])
      
# Open the first file to get the shape of the data arrays (assuming all files have the same shape)
with h5py.File(os.path.join(loc, time_files[0]), "r") as f:
     u_shape_xlo = f['u_x_1800'].shape 
     xlo = np.array(f['X'])
     ylo = np.array(f['Y'][170:490])
     zlo = np.array(f['Z'])

print(ylo)

ylo = ylo - ylo[0]

print(u_shape_xlo)
print(ylo.shape, zlo.shape)

# Pre-allocate the NumPy arrays for u, v, w
u_data_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))
v_data_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))
w_data_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))
thetal_data_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))

T_data_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))
qc_xlo = np.zeros((num_files, 1, ylo.shape[0], zlo.shape[0]))

[1705. 1715. 1725. 1735. 1745. 1755. 1765. 1775. 1785. 1795. 1805. 1815.
 1825. 1835. 1845. 1855. 1865. 1875. 1885. 1895. 1905. 1915. 1925. 1935.
 1945. 1955. 1965. 1975. 1985. 1995. 2005. 2015. 2025. 2035. 2045. 2055.
 2065. 2075. 2085. 2095. 2105. 2115. 2125. 2135. 2145. 2155. 2165. 2175.
 2185. 2195. 2205. 2215. 2225. 2235. 2245. 2255. 2265. 2275. 2285. 2295.
 2305. 2315. 2325. 2335. 2345. 2355. 2365. 2375. 2385. 2395. 2405. 2415.
 2425. 2435. 2445. 2455. 2465. 2475. 2485. 2495. 2505. 2515. 2525. 2535.
 2545. 2555. 2565. 2575. 2585. 2595. 2605. 2615. 2625. 2635. 2645. 2655.
 2665. 2675. 2685. 2695. 2705. 2715. 2725. 2735. 2745. 2755. 2765. 2775.
 2785. 2795. 2805. 2815. 2825. 2835. 2845. 2855. 2865. 2875. 2885. 2895.
 2905. 2915. 2925. 2935. 2945. 2955. 2965. 2975. 2985. 2995. 3005. 3015.
 3025. 3035. 3045. 3055. 3065. 3075. 3085. 3095. 3105. 3115. 3125. 3135.
 3145. 3155. 3165. 3175. 3185. 3195. 3205. 3215. 3225. 3235. 3245. 3255.
 3265. 3275. 3285. 3295. 3305. 3315. 3325. 3335. 33

In [5]:
# collecting yhi data: For xz plane
datafile = os.path.join(loc, time_files[0])

# Open the first file to get the shape of the data arrays (assuming all files have the same shape)
with h5py.File(os.path.join(loc, time_files[0]), "r") as f:
     u_shape_yhi = f['u_y_4800'].shape 
     xhi = np.array(f['X'][170:490])
     yhi = np.array(f['Y'])
     zhi = np.array(f['Z'])

print(xhi)

xhi = xhi - xhi[0]

print(u_shape_yhi)
print(xhi.shape, zhi.shape)

# Pre-allocate the NumPy arrays for u, v, w
u_data_yhi = np.zeros((num_files, xhi.shape[0], 1, zhi.shape[0]))
v_data_yhi = np.zeros((num_files, xhi.shape[0], 1, zhi.shape[0]))
w_data_yhi = np.zeros((num_files, xhi.shape[0], 1, zhi.shape[0]))
thetal_data_yhi = np.zeros((num_files, xhi.shape[0], 1, zhi.shape[0]))

[1705. 1715. 1725. 1735. 1745. 1755. 1765. 1775. 1785. 1795. 1805. 1815.
 1825. 1835. 1845. 1855. 1865. 1875. 1885. 1895. 1905. 1915. 1925. 1935.
 1945. 1955. 1965. 1975. 1985. 1995. 2005. 2015. 2025. 2035. 2045. 2055.
 2065. 2075. 2085. 2095. 2105. 2115. 2125. 2135. 2145. 2155. 2165. 2175.
 2185. 2195. 2205. 2215. 2225. 2235. 2245. 2255. 2265. 2275. 2285. 2295.
 2305. 2315. 2325. 2335. 2345. 2355. 2365. 2375. 2385. 2395. 2405. 2415.
 2425. 2435. 2445. 2455. 2465. 2475. 2485. 2495. 2505. 2515. 2525. 2535.
 2545. 2555. 2565. 2575. 2585. 2595. 2605. 2615. 2625. 2635. 2645. 2655.
 2665. 2675. 2685. 2695. 2705. 2715. 2725. 2735. 2745. 2755. 2765. 2775.
 2785. 2795. 2805. 2815. 2825. 2835. 2845. 2855. 2865. 2875. 2885. 2895.
 2905. 2915. 2925. 2935. 2945. 2955. 2965. 2975. 2985. 2995. 3005. 3015.
 3025. 3035. 3045. 3055. 3065. 3075. 3085. 3095. 3105. 3115. 3125. 3135.
 3145. 3155. 3165. 3175. 3185. 3195. 3205. 3215. 3225. 3235. 3245. 3255.
 3265. 3275. 3285. 3295. 3305. 3315. 3325. 3335. 33

In [6]:
# Populate the arrays with data from each file
for idx, fname in enumerate(time_files):
    with h5py.File(os.path.join(loc, fname), "r") as f:
        u_data_xlo[idx] = f['u_x_1800'][0,170:490,:] + 6.22
        source_3d = f['u_y_1800'][0,170:490,:].reshape(320, 1, 150)
        u_data_yhi[idx] = source_3d + 6.22
print("u done")

u done


In [7]:
# Populate the arrays with data from each file
for idx, fname in enumerate(time_files):
    with h5py.File(os.path.join(loc, fname), "r") as f:
        v_data_xlo[idx] = f['v_x_1800'][0,170:490,:] - 4.8
        source_3d = f['v_y_1800'][0,170:490,:].reshape(320, 1, 150)
        v_data_yhi[idx] = source_3d - 4.8

        w_data_xlo[idx] = f['w_x_1800'][0,170:490,:]
        source_3d = f['w_y_1800'][0,170:490,:].reshape(320, 1, 150)
        w_data_yhi[idx] = source_3d

        thetal_data_xlo[idx] = f['thetal_x_1800'][0,170:490,:]
        source_3d = f['thetal_y_1800'][0,170:490,:].reshape(320, 1, 150)
        thetal_data_yhi[idx] = source_3d
        
print("v done")

v done


In [ ]:
# # Populate the arrays with data from each file
# for idx, fname in enumerate(time_files):
#     with h5py.File(os.path.join(loc, fname), "r") as f:
#         w_data_xlo[idx] = f['w_x_1800'][0,170:490,:]
#         source_3d = f['w_y_1800'][0,170:490,:].reshape(320, 1, 150)
#         w_data_yhi[idx] = source_3d
# print("w done")

In [ ]:
# # Populate the arrays with data from each file
# for idx, fname in enumerate(time_files):
#     with h5py.File(os.path.join(loc, fname), "r") as f:
#         thetal_data_xlo[idx] = f['thetal_x_1800'][0,170:490,:]
#         source_3d = f['thetal_y_1800'][0,170:490,:].reshape(320, 1, 150)
#         thetal_data_yhi[idx] = source_3d
# print("thetal done")

In [1]:
thetal_data_yhi

NameError: name 'thetal_data_yhi' is not defined

In [8]:
# AMR-Wind domain (xlo):
ylo_aw, zlo_aw = 0, 0
yhi_aw, zhi_aw =  3200, 1520

ny = 320
nz = 152

dy, dz = (yhi_aw-ylo_aw)/ny, (zhi_aw-zlo_aw)/nz
print(dy, dz)

# The desired output at cell centers
y_out = np.linspace(ylo_aw + dy/2, yhi_aw-dy/2, ny, endpoint=True)
z_out = np.linspace(zlo_aw + dz/2, zhi_aw-dy/2, nz, endpoint=True)
Y, Z = np.meshgrid(y_out, z_out, indexing='ij')

# Create arrays with proper dimensions and populate data with arrays
velocity_bc_xlo = np.zeros((num_files, ny, nz, 3))
temperature_bc_xlo = np.zeros((num_files, ny, nz))
ptemperature_bc_xlo = np.zeros((num_files, ny, nz))
tke_bc_xlo = np.ones((num_files, ny, nz)) * 0.001 # Initialize to small value

for nt in range(num_files):

    # # Build a 2D interpolator from PINACLES data
    u_int = interpolate.RegularGridInterpolator((ylo, zlo), u_data_xlo[nt, 0, :,:], method='linear', bounds_error=False, fill_value=None)
    v_int = interpolate.RegularGridInterpolator((ylo, zlo), v_data_xlo[nt, 0, :,:], method='linear', bounds_error=False, fill_value=None)
    w_int = interpolate.RegularGridInterpolator((ylo, zlo), w_data_xlo[nt, 0, :,:], method='linear', bounds_error=False, fill_value=None)
    thetal_int = interpolate.RegularGridInterpolator((ylo, zlo), thetal_data_xlo[nt, 0, :,:], method='linear', bounds_error=False, fill_value=None)

    # Interpolate the data 
    velocity_bc_xlo[nt, :, :, 0] = u_int((Y, Z))
    velocity_bc_xlo[nt, :, :, 1] = v_int((Y, Z))
    velocity_bc_xlo[nt, :, :, 2] = w_int((Y, Z))

    # assign dry potential temperature
    ptemperature_bc_xlo[nt, :, :] = thetal_int((Y, Z))

10.0 10.0


In [9]:
# AMR-Wind domain (yhi):
xlo_aw, zlo_aw = 0, 0
xhi_aw, zhi_aw =  3200, 1520

nx = 320
nz = 152

dx, dz = (xhi_aw-xlo_aw)/nx, (zhi_aw-zlo_aw)/nz
print(dx, dz)

# The desired output at cell centers
x_out = np.linspace(xlo_aw + dx/2, xhi_aw-dx/2, nx, endpoint=True)
z_out = np.linspace(zlo_aw + dz/2, zhi_aw-dy/2, nz, endpoint=True)
X, Z = np.meshgrid(x_out, z_out, indexing='ij')

# Create arrays with proper dimensions and populate data with arrays
velocity_bc_yhi = np.zeros((num_files, nx, nz, 3))
temperature_bc_yhi = np.zeros((num_files, nx, nz))
ptemperature_bc_yhi = np.zeros((num_files, nx, nz))
tke_bc_yhi = np.ones((num_files, nx, nz)) * 0.001 # Initialize to small value

for nt in range(num_files):

    # # Build a 2D interpolator from PINACLES data
    u_int = interpolate.RegularGridInterpolator((xhi, zhi), u_data_yhi[nt, :, 0,:], method='linear', bounds_error=False, fill_value=None)
    v_int = interpolate.RegularGridInterpolator((xhi, zhi), v_data_yhi[nt, :, 0,:], method='linear', bounds_error=False, fill_value=None)
    w_int = interpolate.RegularGridInterpolator((xhi, zhi), w_data_yhi[nt, :, 0,:], method='linear', bounds_error=False, fill_value=None)
    thetal_int = interpolate.RegularGridInterpolator((xhi, zhi), thetal_data_yhi[nt, :, 0,:], method='linear', bounds_error=False, fill_value=None)

    # Interpolate the data 
    velocity_bc_yhi[nt, :, :, 0] = u_int((Y, Z))
    velocity_bc_yhi[nt, :, :, 1] = v_int((Y, Z))
    velocity_bc_yhi[nt, :, :, 2] = w_int((Y, Z))

    # assign dry potential temperature
    ptemperature_bc_yhi[nt, :, :] = thetal_int((Y, Z))

10.0 10.0


In [14]:
outfile = '/scratch/sbidadi/oracle/cloud_data/bc_amr_wind_res_10m_ar_1_dry_theta_more_data_xlo_yhi.nc'

'''
This is the output file that will be saved with the 
boundary condition for amr-wind
'''
with Dataset(outfile, "w", format="NETCDF4") as rootgrp:
    ## ~~~~~~~~ Operate on the root group ~~~~~~~~
    # Add dimensions to the root group
    rootgrp.createDimension('sdim', 1)
    rootgrp.createDimension('pdim', 2)
    rootgrp.createDimension('vdim', 3)
    rootgrp.createDimension('nt', None)

    # Add groups
    grp_xlo = rootgrp.createGroup("/xlo")
    grp_yhi = rootgrp.createGroup("/yhi")
    
    # Add variables    
    rootgrp.createVariable("time","f8",("nt",))
    rootgrp['time'][:] = times_in_seconds


    ## ~~~~~~~~ Operate on the xlo group ~~~~~~~~
    # Add groups
    grp_xlo_l0 = grp_xlo.createGroup('/level_0')

    # Add variables
    grp_xlo.createVariable("normal","i4",)
    grp_xlo.createVariable("side","i4",)
    grp_xlo.createVariable("perpendicular","i4",("pdim",))

    grp_xlo['normal'][:] = 0
    grp_xlo['side'][:] = 0
    grp_xlo['perpendicular'][:] = [1, 2]

    ## ~~~~~~~~ Operate on the xlo/level_0 ~~~~~~~~
    # Add dimensions
    grp_xlo_l0.createDimension('nx', 1)
    grp_xlo_l0.createDimension('ny', ny)
    grp_xlo_l0.createDimension('nz', nz)

    # Add variables
    grp_xlo_l0.createVariable("lengths","f8",("pdim",))
    grp_xlo_l0.createVariable("lo","f8",("pdim",))
    grp_xlo_l0.createVariable("hi","f8",("pdim",))
    grp_xlo_l0.createVariable("dx","f8",("pdim",))
    grp_xlo_l0.createVariable("velocity","f8",("nt", "ny", "nz", "vdim",))
    grp_xlo_l0.createVariable("temperature","f8",("nt", "ny", "nz",))
    grp_xlo_l0.createVariable("tke","f8",("nt", "ny", "nz",))  # TODO: Am I precribing SGS TKE correctly?

    print(velocity_bc_xlo.shape)
    print(dy, dz)
    grp_xlo_l0['lengths'][:] = [yhi_aw-ylo_aw, zhi_aw-zlo_aw]
    grp_xlo_l0['lo'][:] = [ylo_aw, zlo_aw]
    grp_xlo_l0['hi'][:] = [yhi_aw, zhi_aw]
    grp_xlo_l0['dx'][:] = [dy, dz]
    grp_xlo_l0['velocity'][:] = velocity_bc_xlo
    grp_xlo_l0['temperature'][:] = ptemperature_bc_xlo
    grp_xlo_l0['tke'][:] = tke_bc_xlo

    print(grp_xlo_l0['velocity'])


    ## ~~~~~~~~ Operate on the yhi group ~~~~~~~~
    # Add groups
    grp_yhi_l0 = grp_yhi.createGroup('/level_0')

    # Add variables
    grp_yhi.createVariable("normal","i4",)
    grp_yhi.createVariable("side","i4",)
    grp_yhi.createVariable("perpendicular","i4",("pdim",))

    grp_yhi['normal'][:] = 1
    grp_yhi['side'][:] = 1
    grp_yhi['perpendicular'][:] = [0, 2]

    ## ~~~~~~~~ Operate on the yhi/level_0 ~~~~~~~~
    # Add dimensions
    grp_yhi_l0.createDimension('nx', nx)
    grp_yhi_l0.createDimension('ny', 1)
    grp_yhi_l0.createDimension('nz', nz)

    # Add variables
    grp_yhi_l0.createVariable("lengths","f8",("pdim",))
    grp_yhi_l0.createVariable("lo","f8",("pdim",))
    grp_yhi_l0.createVariable("hi","f8",("pdim",))
    grp_yhi_l0.createVariable("dx","f8",("pdim",))
    grp_yhi_l0.createVariable("velocity","f8",("nt", "nx", "nz", "vdim",))
    grp_yhi_l0.createVariable("temperature","f8",("nt", "nx", "nz",))
    grp_yhi_l0.createVariable("tke","f8",("nt", "nx", "nz",))  # TODO: Am I precribing SGS TKE correctly?

    print(velocity_bc_yhi.shape)
    print(dy, dz)
    grp_yhi_l0['lengths'][:] = [xhi_aw-xlo_aw, zhi_aw-zlo_aw]
    grp_yhi_l0['lo'][:] = [xlo_aw, zlo_aw]
    grp_yhi_l0['hi'][:] = [xhi_aw, zhi_aw]
    grp_yhi_l0['dx'][:] = [dx, dz]
    grp_yhi_l0['velocity'][:] = velocity_bc_yhi
    grp_yhi_l0['temperature'][:] = ptemperature_bc_yhi
    grp_yhi_l0['tke'][:] = tke_bc_yhi

    print(grp_yhi_l0['velocity'])

(2161, 320, 152, 3)
10.0 10.0
<class 'netCDF4._netCDF4.Variable'>
float64 velocity(nt, ny, nz, vdim)
path = /xlo/level_0
unlimited dimensions: nt
current shape = (2161, 320, 152, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
(2161, 320, 152, 3)
10.0 10.0
<class 'netCDF4._netCDF4.Variable'>
float64 velocity(nt, nx, nz, vdim)
path = /yhi/level_0
unlimited dimensions: nt
current shape = (2161, 320, 152, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
